In [1]:
!pip install -q --upgrade pip
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.3/539.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.9/704.9 k

In [ ]:
!pip uninstall jaxlib
!pip install jaxlib==0.4.1

Found existing installation: jaxlib 0.3.25+cuda11.cudnn805
Uninstalling jaxlib-0.3.25+cuda11.cudnn805:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/jaxlib-0.3.25+cuda11.cudnn805.dist-info/*
    /usr/local/lib/python3.8/dist-packages/jaxlib/*
Proceed (Y/n)? 

In [ ]:
import numpy
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader


def r_power_adjacency_matrix(A, r):
  """
  Calculate the r-power for adjacency matrix A.
  From a 3d tensor with batch dimension
  """
  last_As = A
  for i in range(r-1):
    next_As = np.matmul(last_As, A)
    next_As = next_As + last_As
    next_As = next_As.at[next_As != 0].set(1)
    last_As = next_As
  
  return next_As

def r_power_adjacency_matrix_list(As, r):
  """
  Calculate the r-power for adjacency matrix A.
  From a list of 2d arrays
  """
  out = []
  for A in As:
    last_As = A
    for i in range(r-1):
      next_As = np.matmul(last_As, A)
      next_As = next_As + last_As
      next_As = next_As.at[next_As != 0].set(1)
      last_As = next_As
    out.append(last_As)
  
  return out

def neigbourhood_intersections(A):
  """
  Interpretation of out:
  out[0,1,3] if intersect we
  intersect the nodes we can reach from 1 and
  from 2 can we reach node 3?
  """
  out = np.zeros((A.shape[0], A.shape[1], A.shape[1], A.shape[1]), 
                 dtype="int32")
  for k in range(A.shape[0]):
    A1 = np.full((A.shape[1], A.shape[1], A.shape[1]), A[k,:])
    A2 = np.swapaxes(A1, 0, 1)

    out = out.at[k,:].set(np.array(np.logical_and(A1, A2), dtype="int32"))

  return out

def neigbourhood_intersections_list(As):
  """
  Interpretation of out:
  out[0,1,3] if intersect we
  intersect the nodes we can reach from 1 and
  from 2 can we reach node 3?
  """
  out = list()
  for A in As:
    A1 = np.full((A.shape[1], A.shape[1], A.shape[1]), A)
    A2 = np.swapaxes(A1, 0, 1)

    out.append(np.array(np.logical_and(A1, A2), dtype="int32"))

  return out

def set_lower_tri_zero(A):
  #for i in range(A.shape[0]):
  lower_ind = np.tril_indices_from(A, -1)
  A = A.at[lower_ind[0], lower_ind[1]].set(0)
  return A

def cumsum_from_piecewise(a):
  """
  cumsum_from_full(np.array([0,0,1,1,2,2,3,3,3,4]))
  = [ 0.,  0.,  1.,  1.,  3.,  3.,  6.,  6.,  6., 10.]
  """
  out_tmp = np.zeros((a[1:].shape))
  out = np.zeros((a.shape))
  ind = a[:-1] != a[1:]
  out_tmp = out_tmp.at[ind].set(a[1:][ind])
  out = out.at[0].set(a[1])
  out = out.at[1:].set(out_tmp)
  out = np.cumsum(out, dtype="int32")
  return out

#Create the initial edge feature Matrix and the reference Matrix:
The edge feature Matrix is of dimension $gamma$ x n. Where n is the size of the node/edge feature and $gamma = \sum_{i}^{\#Batches} \#E_{G_i}$  
The reference Matrix is of dimension $ \beta $ x 3 with 
$\beta = \sum_{b}^{\#Batches} 
\sum_{e_{ij} \in E^{b}_{G^r}}
 \Gamma_G^r(v_i) \cap \Gamma_G^r(v_j) $
 

In [ ]:
def neigbourhood_intersections_single(A):
  """
  Interpretation of out:
  out[0,1,3] if intersect we
  intersect the nodes we can reach from 1 and
  from 2 can we reach node 3?
  """
  A1 = np.full((A.shape[1], A.shape[1], A.shape[1]), A)
  A2 = np.swapaxes(A1, 0, 1)
  return np.array(np.logical_and(A1, A2), dtype="int32")

def f(x,y,n):
  """
  calculate the edge index of elements
  x and y for a upper 
  triangular matrix of shape n x n.
  """
  x_tmp = x.at[x>y].set(y[x>y])
  y = y.at[x>y].set(x[x>y])
  x = x_tmp
  out = x * n + y - (((x+1)**2 - (x+1)) / 2)
  return out

def calculate_reference_matrix(A):
  A_int = neigbourhood_intersections_single(A)
  n = A.shape[0]
  a = np.transpose(np.array(np.nonzero((A_int)))) 
  l = a[:,0]
  i = a[:,1]
  j = a[:,2]
  e_ij = np.expand_dims(np.array(f(i,j,n), dtype="int32"), 1)
  e_il = np.expand_dims(np.array(f(i,l,n), dtype="int32"), 1)
  e_lj = np.expand_dims(np.array(f(l,j,n), dtype="int32"), 1)
  out = np.append(np.append(e_ij, e_il, axis=1), e_lj, axis=1)
  out = out[i>=j]

  return out

def to_dense(node_list, size):
  """
  Naive implementation, to get a
  adjacency matrix from a node list.
  Node list 2xn -> adjacency matrix nxn
  """
  A = np.zeros((size, size))
  node_list = data.edge_index.tolist()
  for i,j in zip(node_list[0], node_list[1]):
    A = A.at[i,j].set(1)
  return A

def zero_append(a, shape):
  """
  Add zero columns and rows to the array 
  a, to make it of shape size x size.
  """
  out = np.zeros((shape[0],shape[1]))
  out = out.at[:a.shape[0],:a.shape[1]].set(a)
  return out

In [ ]:
dataset = TUDataset(root="Masterarbeit", name="MUTAG")

# make lists of dense adjacency matrix, node feature array and y
graps_node_features = list()
As = list()
ys = list()

for data in dataset:
  As.append(to_dense(data.edge_index, len(data.x)))
  graps_node_features.append(np.array(data.x))
  ys.append(np.array(data.y))

In [ ]:
#calculate the adjacenecy matrix for the r-th power of the graphs
As_2 = r_power_adjacency_matrix_list(As, 2)
As_5 = r_power_adjacency_matrix_list(As, 5)

In [ ]:
# the reference matrix

# reference matrix for each sample in the batch
R_list = [calculate_reference_matrix(A) for A in As_2]

# merge the reference matrix from the samples into a 
# big reference matrix containing all samples.
# To do this we need to shift the index in in the 
# referende matrix at position i in the list
# by the cumsum of the indices "used" in the 
# all the ref matricies befor i
R_list_max = [int(x.max())+1 for x in R_list]
R_list_max_cumsum = numpy.array(R_list_max).cumsum().tolist()
R_list_max_cumsum = [0] + R_list_max_cumsum[:-1]
intersection_ind = [x+y for (x,y) in zip(R_list_max_cumsum, R_list)]
R = intersection_ind[0]
for a in intersection_ind[1:]:
  R = np.append(R,a, axis=0)

In [ ]:
# get the initial edge feature matrix
# when the graph specification has only node features
graphs_edge_features_l = list()
for grap_node_features in graps_node_features:
  n = grap_node_features.shape[0]
  m = grap_node_features.shape[1]
  # the edge indices for the diag of a upper diagonal matrix
  ind = np.array(f(np.array(range(n)),np.array(range(n)),n), dtype="int32")
  graph_edge_features = np.zeros((int(ind[-1])+1, m))
  graph_edge_features = graph_edge_features.at[ind,:].set(grap_node_features)
  graphs_edge_features_l.append(graph_edge_features)

graphs_edge_features = graphs_edge_features_l[0]
for a in graphs_edge_features_l[1:]:
  graphs_edge_features = np.append(graphs_edge_features,a, axis=0)

In [ ]:
x_gamma_1 = np.take(graphs_edge_features, R[:,1], axis=0)
x_gamma_2 = np.take(graphs_edge_features, R[:,2], axis=0)
x_gamma = x_gamma_1 + x_gamma_2
X_gamma_sum = jax.ops.segment_sum(x_gamma, R[:,0])

graphs_edge_features.shape[0] - X_gamma_sum.shape[0]

In [ ]:
X_gamma_sum

In [ ]:
x_gamma_1 = np.take(graphs_edge_features, R[:,1], axis=0)
x_gamma_2 = np.take(graphs_edge_features, R[:,2], axis=0)
x_gamma_1 = np.expand_dims(x_gamma_1, 1)
x_gamma_2 = np.expand_dims(x_gamma_2, 1)
x_gamma_3 = np.append(x_gamma_1,x_gamma_1, 1)

x_gamma_4 = np.sum(x_gamma_3, 1)
X_gamma_5 = jax.ops.segment_sum(x_gamma, R[:,0])


In [ ]:
x_gamma_3

In [60]:
from typing import Callable, Iterable, Optional, Sequence, Tuple, Union
from neural_tangents import Kernel
from neural_tangents._src.stax.requirements import Bool, Diagonal, get_diagonal_outer_prods, layer, mean_and_var, requires, supports_masking

@layer
@supports_masking(remask_kernel=False)
def nn_aggregation():
  """
  Return a layer, that implements the gatter
  and scatter operations given the reference Matrix 
  R.
  R is of shape (#graps in the batch *
  # edges of each grap) x 2
  """
  init_fn = lambda rng, input_shape: (input_shape, ())

  def apply_fn(params,
               inputs: np.ndarray,
               *,
               pattern: Optional[np.ndarray] = None,
               **kwargs):
    
    x_gamma_1 = np.take(graphs_edge_features, pattern[:,1], axis=0)
    x_gamma_2 = np.take(graphs_edge_features, pattern[:,2], axis=0)
    x_gamma_1_2 = np.append(np.expand_dims(x_gamma_1, 1), np.expand_dims(x_gamma_2, 1), 1)
    x_gamma_1_2 = np.sum(x_gamma_1_2, 1)
    # TODO: We need to specifie the num_segments argument staticaly
    # But, which value should we use?
    X_gamma_sum = jax.ops.segment_sum(x_gamma_1_2, pattern[:,0], np.max(pattern[:,0]) + 1)
    return X_gamma_sum

  def kernel_fn(k: Kernel,
                *,
                pattern: Tuple[Optional[np.ndarray],
                               Optional[np.ndarray]] = (None, None),
                **kwargs):
    return k
    

  return init_fn, apply_fn, kernel_fn

In [61]:
# 2-WL operations including weights

x = graphs_edge_features[:10,:]

out_dim = 7

L_branche = stax.serial(
    stax.Dense(out_dim),
    nn_aggregation(),
    )

Gamma_branche = stax.serial(
    stax.Dense(out_dim),       
)

init_fn, apply_fn, kernel_fn = stax.serial(
    stax.FanOut(2), 
    stax.parallel(L_branche, Gamma_branche), 
    stax.FanInSum(),
    stax.Relu(),
    )


out = apply_fn((), x)

kernel = kernel_fn(x, x, 'ntk', pattern = R[:10,:])

ConcretizationTypeError: ignored

In [24]:
kernel.shape

(10, 10)